In [20]:
import pandas as pd
import ee
#import geetools
import folium
import time
from folium import plugins
##import Leaf_tools
import math
from collections import defaultdict
import numpy as np
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AY0e-g6LaB1ZXuWQMrWBmFswEpB004zDS-TE_Si8V68UsAXjRJq16lWNBT0



Successfully saved authorization token.


**Import data**

In [21]:
tiles_CCRS = ee.FeatureCollection("users/ganghong/TilesSampling_Sub")
print (tiles_CCRS.size().getInfo())
tiles_Bel = ee.FeatureCollection('users/ganghong/TilesSampling_Belmanip_Sub')
print (tiles_Bel.size().getInfo())
ezone = ee.FeatureCollection("users/ganghong/NA_Forests")
fapar_CCRS = ee.ImageCollection("users/ganghong/FAPAR_CCRS")
fcover_CCRS = ee.ImageCollection("users/ganghong/FCOVER_CCRS")
LAI_CCRS = ee.ImageCollection("users/ganghong/LAI_CCRS")
fapar_Bel = ee.ImageCollection("users/ganghong/FAPAR_Bel")
fcover_Bel = ee.ImageCollection("users/ganghong/FCOVER_Bel")
LAI_Bel = ee.ImageCollection("users/ganghong/LAI_Bel")
##Define Forest Cover map to use - should be a global constant
NALCMS_forest = ee.ImageCollection('users/rfernand387/NA_NALCMS_2015_tiles')\
                  .map(lambda image:image.select("b1").rename("landcover"))\
                  .merge(ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V/Global")\
                  .map(lambda image:image.select("discrete_classification").remap([0,20,30,40,50,60,70,80,90,100,111,112,113,114,115,116,121,122,123,124,125,126,200],[0,8,10,15,17,16,19,18,14,13,1,3,1,5,6,6,2,4,2,5,6,6,18],0).toUint8().rename("landcover")))

NALCMS_water = ee.ImageCollection('users/rfernand387/NA_NALCMS_2015_tiles')\
                 .map(lambda image:image.select("b1").rename("partition"))\
                 .merge(ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V/Global")\
                 .map(lambda image:image.select("discrete_classification")\
                                                    .remap([0,20,30,40,50,60,70,80,90,100,111,112,113,114,115,116,121,122,123,124,125,126,200],[0,8,10,15,17,16,19,18,14,13,1,3,1,5,6,6,2,4,2,5,6,6,18],0)\
                                                    .toUint8()\
                                                    .rename("partition")))\
                .mosaic()\
                .remap([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19],[ 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,  0,  0, 0, 1, 0, 0, 0, 1, 0],0,"partition")\
                  .rename('waterCover')                            

# print (fapar_CCRS.size().getInfo())
# print (fcover_CCRS.size().getInfo())
# print (LAI_CCRS.size().getInfo())
# print (fapar_Bel.size().getInfo())
# print (fcover_Bel.size().getInfo())
# print (LAI_Bel.size().getInfo())

28
35


In [17]:

# def Leaf_tools(mapBounds, startDate, endDate,tilename, outputName):
#     def check_ee_tasks(ee_tasks: list = []):
#         for task in ee_tasks:
#             taskStatus = ee.data.getTaskStatus(task.id)[0]
#             print(taskStatus["description"] + ": " + taskStatus["state"])

#     # Wait loop for Earth Engine tasks to complete. Polls for the task status the specificed number of seconds until it is no longer active
#     def task_wait_loop(ee_task, wait_interval):
#         print(ee.data.getTaskStatus(ee_task.id)[0]["description"] + ":", end=" ")
#         prev_task_status = ee.data.getTaskStatus(ee_task.id)[0]["state"]
#         print(prev_task_status, end=" ")
#         while ee_task.active():
#             task_status = ee.data.getTaskStatus(ee_task.id)[0]["state"]
#             if (task_status != prev_task_status):
#                 print(task_status, end=" ")
#             prev_task_status = task_status
#             time.sleep(wait_interval)
#         print(ee.data.getTaskStatus(ee_task.id)[0]["state"])    

#     def s2_createFeatureCollection_estimates():
#         return ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/s2_sl2p_weiss_or_prosail_NNT3_Single_0_1')

#     def s2_createFeatureCollection_errors():
#         return ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/s2_sl2p_weiss_or_prosail_NNT3_Single_0_1_Error')

#     def s2_createFeatureCollection_domains():
#         return ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/weiss_or_prosail3_NNT3_Single_0_1_DOMAIN')

#     def s2_createFeatureCollection_range():
#         return ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/weiss_or_prosail3_NNT3_Single_0_1_RANGE')

#     def s2_createFeatureCollection_Network_Ind():
#         return ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/Parameter_file_sl2p')

#     def s2_createImageCollection_partition():
#         return ee.ImageCollection('users/rfernand387/NA_NALCMS_2015_tiles').map(
#             lambda image: image.select("b1").rename("partition")).merge(
#             ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V/Global").map(
#                 lambda image: image.select("discrete_classification").remap(
#                     [0, 20, 30, 40, 50, 60, 70, 80, 90, 100, 111, 112, 113, 114, 115, 116, 121, 122, 123, 124, 125, 126,
#                      200], [0, 8, 10, 15, 17, 16, 19, 18, 14, 13, 1, 3, 1, 5, 6, 6, 2, 4, 2, 5, 6, 6, 18],
#                     0).toUint8().rename("partition")))

#     def s2_createFeatureCollection_legend():
#         return ee.FeatureCollection('users/rfernand387/COPERNICUS_S2_SR/Legend_sl2p')

#     def l8_createFeatureCollection_estimates():
#         return ee.FeatureCollection('users/rfernand387/LANDSAT_LC08_C01_T1_SR_SL2P_OUTPUT')

#     def l8_createFeatureCollection_errors():
#         return ee.FeatureCollection('users/rfernand387/LANDSAT_LC08_C01_T1_SR_SL2P_ERRORS')

#     def l8_createFeatureCollection_domains():
#         return ee.FeatureCollection('users/rfernand387/LANDSAT_LC08_C01_T1_SR/LANDSAT_LC08_C01_T1_SR_DOMAIN')

#     def l8_createFeatureCollection_range():
#         return ee.FeatureCollection('users/rfernand387/LANDSAT_LC08_C01_T1_SR/LANDSAT_LC08_C01_T1_SR_RANGE')

#     def l8_createFeatureCollection_Network_Ind():
#         return ee.FeatureCollection('users/rfernand387/LANDSAT_LC08_C01_T1_SR/Parameter_file_sl2p')

#     def l8_createImageCollection_partition():
#         return ee.ImageCollection('users/rfernand387/NA_NALCMS_2015_tiles').map(
#             lambda image: image.select("b1").rename("partition")).merge(
#             ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V/Global").map(
#                 lambda image: image.select("discrete_classification").remap(
#                     [0, 20, 30, 40, 50, 60, 70, 80, 90, 100, 111, 112, 113, 114, 115, 116, 121, 122, 123, 124, 125, 126,
#                      200], [0, 8, 10, 15, 17, 16, 19, 18, 14, 13, 1, 3, 1, 5, 6, 6, 2, 4, 2, 5, 6, 6, 18],
#                     0).toUint8().rename("partition")))

#     def l8_createFeatureCollection_legend():
#         return ee.FeatureCollection('users/rfernand387/LANDSAT_LC08_C01_T1_SR/Legend_sl2p')

#     # add a 'date' band: number of days since epoch
#     def addDate(image):
#         return image.addBands(
#             ee.Image.constant(ee.Date(image.date()).millis().divide(86400000)).rename('date').toUint16())

#     # computes a delta time property for an image
#     def deltaTime(midDate, image):
#         return ee.Image(image.set("deltaTime", ee.Number(image.date().millis()).subtract(ee.Number(midDate)).abs()))

#     # mask pixels that are not clear sky in a S2 MSI image
#     def s2MaskClear(image):
#         qa = image.select('QA60');
#         mask = qa.bitwiseAnd(1 << 10).eq(0).And(qa.bitwiseAnd(1 << 11).eq(0));
#         return image.updateMask(mask)

#     # add s2 geometry bands scaled by 10000
#     def addS2Geometry(colOptions, image):
#         return (image.addBands(
#             ee.Image.constant(0).multiply(3.1415).divide(180).cos().multiply(10000).toUint16().rename(['cosVZA']))
#                 .addBands(
#             image.metadata(colOptions["sza"]).multiply(3.1415).divide(180).cos().multiply(10000).toUint16().rename(
#                 ['cosSZA']))
#                 .addBands(
#             image.metadata(colOptions["saa"]).subtract(image.metadata(colOptions["saa"])).multiply(3.1415).divide(
#                 180).cos().multiply(10000).toUint16().rename(['cosRAA'])));

#     # sentinel 2 land mask
#     def s2MaskLand(image):
#         return image.updateMask((image.select('SCL').eq(4)).Or(image.select('SCL').eq(5)))

#     # returns image with selected bands scaled
#     def scaleBands(bandList, scaleList, image):
#         bandList = ee.List(bandList)
#         scaleList = ee.List(scaleList)
#         return image.addBands(srcImg=image.select(bandList).multiply(ee.Image.constant(scaleList)).rename(bandList),
#                               overwrite=True)

#     # Determine if inputs fall in domain of algorithm
#     # Need to be updated to allow for the domain to vary with partition
#     def invalidInput(sl2pDomain, bandList, image):

#         sl2pDomain = ee.FeatureCollection(sl2pDomain).aggregate_array("DomainCode").sort()
#         bandList = ee.List(bandList).slice(3)
#         image = ee.Image(image)

#         # code image bands into a single band and compare to valid codes to make QC band
#         image = image.addBands(image.select(bandList)
#                                .multiply(ee.Image.constant(ee.Number(10)))
#                                .ceil()
#                                .mod(ee.Number(10))
#                                .uint8()
#                                .multiply(ee.Image.constant(
#             ee.List.sequence(0, bandList.length().subtract(1)).map(lambda value: ee.Number(10).pow(ee.Number(value)))))
#                                .reduce("sum")
#                                .remap(sl2pDomain, ee.List.repeat(0, sl2pDomain.length()), 1)
#                                .rename("QC"))
#         return image

#     # returns image with single band named network id corresponding given
#     def makeIndexLayer(image, legend, Network_Ind):

#         image = ee.Image(image)  # partition image
#         legend = ee.FeatureCollection(legend)  # legend to convert partition numbers to networks
#         Network_Ind = ee.FeatureCollection(Network_Ind)  # legend to convert networks to networkIDs

#         # get lists of valid partitions
#         legend_list = legend.toList(legend.size())
#         landcover = legend_list.map(lambda feature: ee.Feature(feature).getNumber('Value'))

#         # get corresponding networkIDs
#         networkIDs = legend_list.map(lambda feature: ee.Feature(feature).get('SL2P Network')) \
#             .map(lambda propertyValue: ee.Feature(ee.FeatureCollection(Network_Ind).first()).toDictionary().getNumber(
#             propertyValue))

#         return image.remap(landcover, networkIDs, 0).rename('networkID')

#     # Read coefficients of a network from csv EE asset
#     def getCoefs(netData, ind):
#         return ((ee.Feature(netData)).getNumber(ee.String('tabledata').cat(ee.Number(ind).int().format())))

#     # Parse one row of CSV file for a network into a global variable
#     #
#     # We assume a two hidden layer network with tansig functions but
#     # allow for variable nodes per layer
#     def makeNets(feature, M):

#         feature = ee.List(feature);
#         M = ee.Number(M);

#         # get the requested network and initialize the created network
#         netData = ee.Feature(feature.get(M.subtract(1)));
#         net = {};

#         # input slope
#         num = ee.Number(6);
#         start = num.add(1);
#         end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())));
#         net["inpSlope"] = ee.List.sequence(start, end).map(lambda ind: getCoefs(netData, ind))

#         # input offset
#         num = end.add(1)
#         start = num.add(1)
#         end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())))
#         net["inpOffset"] = ee.List.sequence(start, end).map(lambda ind: getCoefs(netData, ind))

#         # hidden layer 1 weight
#         num = end.add(1)
#         start = num.add(1)
#         end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())))
#         net["h1wt"] = ee.List.sequence(start, end).map(lambda ind: getCoefs(netData, ind))

#         # hidden layer 1 bias
#         num = end.add(1)
#         start = num.add(1)
#         end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())))
#         net["h1bi"] = ee.List.sequence(start, end).map(lambda ind: getCoefs(netData, ind))

#         # hidden layer 2 weight
#         num = end.add(1)
#         start = num.add(1)
#         end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())))
#         net["h2wt"] = ee.List.sequence(start, end).map(lambda ind: getCoefs(netData, ind))

#         # hidden layer 2 bias
#         num = end.add(1)
#         start = num.add(1)
#         end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())))
#         net["h2bi"] = ee.List.sequence(start, end).map(lambda ind: getCoefs(netData, ind))

#         # output slope
#         num = end.add(1)
#         start = num.add(1)
#         end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())))
#         net["outSlope"] = ee.List.sequence(start, end).map(lambda ind: getCoefs(netData, ind))

#         # output offset
#         num = end.add(1)
#         start = num.add(1)
#         end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())))
#         net["outBias"] = ee.List.sequence(start, end).map(lambda ind: getCoefs(netData, ind))

#         return (ee.Dictionary(net))

#     # Parse CSV file with list of networks for a selected variable
#     # This will parse one network for each landclass partition
#     def makeNetVars(asset, numNets, variableNum):

#         asset = ee.FeatureCollection(asset)
#         numNets = ee.Number(numNets)
#         variableNum = ee.Number(variableNum)

#         # get selected network
#         list_features = asset.flatten()
#         filtered_features = ee.FeatureCollection(asset.filter(ee.Filter.eq('tabledata3', variableNum))).toList(numNets)

#         return ee.List.sequence(1, numNets).map(lambda netNum: makeNets(filtered_features, netNum))

#     # returns dictionary with image masked so the networkID band equals the netIndex and the corresponding network
#     def selectNet(image, netList, inputNames, netIndex):

#         image = ee.Image(image)
#         netList = ee.List(netList)
#         inputNames = ee.List(inputNames)
#         netIndex = ee.Number(netIndex).int()

#         return ee.Dictionary() \
#             .set("Image", ee.Image(image.updateMask(image.select('networkID').eq(netIndex)).select(inputNames))) \
#             .set("Network", ee.List(netList.get(netIndex)))

#         # applies two layer neural network within input and output scaling

#     def applyNet(outputName, netDict):

#         outputName = ee.String(outputName)
#         netDict = ee.Dictionary(netDict)

#         inp = ee.Image(netDict.get('Image'))
#         net = ee.Dictionary(netDict.get('Network'))

#         # Input scaling
#         l1inp2D = inp.multiply(ee.Image(net.toArray(ee.List(['inpSlope']), 0).transpose()) \
#                                .arrayProject([0]) \
#                                .arrayFlatten([inp.bandNames()])) \
#             .add(ee.Image(net.toArray(ee.List(['inpOffset']), 0).transpose()) \
#                  .arrayProject([0]) \
#                  .arrayFlatten([inp.bandNames()]))

#         # Hidden layers
#         l12D = ee.Image(net.toArray(ee.List(['h1wt']), 0).reshape(
#             [ee.List(net.get('h1bi')).length(), ee.List(net.get('inpOffset')).length()])) \
#             .matrixMultiply(l1inp2D.toArray().toArray(1)) \
#             .add(ee.Image(net.toArray(ee.List(['h1bi']), 0).transpose())) \
#             .arrayProject([0]).arrayFlatten([['h1w1', 'h1w2', 'h1w3', 'h1w4', 'h1w5']])

#         # apply tansig 2/(1+exp(-2*n))-1
#         l2inp2D = ee.Image(2).divide(ee.Image(1).add((ee.Image(-2).multiply(l12D)).exp())).subtract(ee.Image(1))

#         # purlin hidden layers
#         l22D = l2inp2D.multiply(ee.Image(net.toArray(ee.List(['h2wt']), 0).transpose()) \
#                                 .arrayProject([0]) \
#                                 .arrayFlatten([['h2w1', 'h2w2', 'h2w3', 'h2w4', 'h2w5']])) \
#             .reduce('sum') \
#             .add(ee.Image(net.toArray(ee.List(['h2bi']), 0))) \
#             .arrayProject([0]) \
#             .arrayFlatten([['h2bi']])

#         # Output scaling
#         outputBand = l22D.subtract(ee.Image(ee.Number(net.get('outBias')))) \
#             .divide(ee.Image(ee.Number(net.get('outSlope'))))

#         # Return network output
#         return (outputBand.rename(outputName))

#     # returns image with single band named networkid corresponding given
#     # input partition image remapped to networkIDs
#     # applies a set of shallow networks to an image based on a provided partition image band
#     def wrapperNNets(network, partition, netOptions, colOptions, suffixName, imageInput):

#         # typecast function parameters
#         network = ee.List(network)
#         partition = ee.Image(partition)
#         netOptions = netOptions
#         colOptions = colOptions
#         suffixName = suffixName
#         imageInput = ee.Image(imageInput)

#         # parse partition  used to identify network to use
#         partition = partition.clip(imageInput.geometry()).select(['partition'])

#         # determine networks based on collection
#         netList = ee.List(network.get(ee.Number(netOptions.get("variable")).subtract(1)));

#         # parse land cover into network index and add to input image
#         imageInput = imageInput.addBands(makeIndexLayer(partition, colOptions["legend"], colOptions["Network_Ind"]))

#         # define list of input names
#         return ee.ImageCollection(ee.List.sequence(0, netList.size().subtract(1)) \
#                                   .map(
#             lambda netIndex: selectNet(imageInput, netList, netOptions["inputBands"], netIndex)) \
#                                   .map(lambda netDict: applyNet(suffixName + outputName, netDict))) \
#             .max() \
#             .addBands(partition) \
#             .addBands(imageInput.select('networkID'))

#     # returns dictionary with image masked so the networkID band equals the netIndex and the corresponding network
#     def selectNet2(image, netList, inputNames, netIndex):

#         image = ee.Image(image)
#         netList = ee.List(netList)
#         inputNames = ee.List(inputNames)
#         netIndex = ee.Number(netIndex).int()
#         result = ee.Dictionary() \
#             .set("Image", ee.Image(image.updateMask(image.select('networkID').eq(netIndex)).select(inputNames))) \
#             .set("Network", ee.List(netList.get(netIndex)))
#         return result

#     # applies two layer neural network within input and output scaling
#     def applyNet2(outputName, netDict):

#         outputName = ee.String(outputName)
#         netDict = ee.Dictionary(netDict)
#         inp = ee.Image(netDict.get('Image'))
#         net = ee.Dictionary(netDict.get('Network'))

#         # Input scaling
#         l1inp2D = inp.multiply(ee.Image(net.toArray(ee.List(['inpSlope']), 0).transpose()) \
#                                .arrayProject([0]) \
#                                .arrayFlatten([inp.bandNames()])) \
#             .add(ee.Image(net.toArray(ee.List(['inpOffset']), 0).transpose()) \
#                  .arrayProject([0]) \
#                  .arrayFlatten([inp.bandNames()]))

#         # Hidden layers
#         l12D = ee.Image(net.toArray(ee.List(['h1wt']), 0).reshape(
#             [ee.List(net.get('h1bi')).length(), ee.List(net.get('inpOffset')).length()])) \
#             .matrixMultiply(l1inp2D.toArray().toArray(1)) \
#             .add(ee.Image(net.toArray(ee.List(['h1bi']), 0).transpose())) \
#             .arrayProject([0]).arrayFlatten([['h1w1', 'h1w2', 'h1w3', 'h1w4', 'h1w5']])

#         # apply tansig 2/(1+exp(-2*n))-1
#         l2inp2D = ee.Image(2).divide(ee.Image(1).add((ee.Image(-2).multiply(l12D)).exp())).subtract(ee.Image(1))

#         # purlin hidden layers
#         l22D = l2inp2D.multiply(ee.Image(net.toArray(ee.List(['h2wt']), 0).transpose()) \
#                                 .arrayProject([0]) \
#                                 .arrayFlatten([['h2w1', 'h2w2', 'h2w3', 'h2w4', 'h2w5']])) \
#             .reduce('sum') \
#             .add(ee.Image(net.toArray(ee.List(['h2bi']), 0))) \
#             .arrayProject([0]) \
#             .arrayFlatten([['h2bi']])

#         # Output scaling
#         outputBand = l22D.subtract(ee.Image(ee.Number(net.get('outBias')))) \
#             .divide(ee.Image(ee.Number(net.get('outSlope'))))

#         # Return network output
#         return (outputBand.rename(outputName))

#     # Read coefficients of a network from csv EE asset
#     def getCoefs2(netData, ind):
#         return ((ee.Feature(netData)).getNumber(ee.String('tabledata').cat(ee.Number(ind).int().format())))

#     # Parse one row of CSV file for a network into a global variable
#     #
#     # We assume a two hidden layer network with tansig functions but
#     # allow for variable nodes per layer
#     def makeNets2(feature, M):

#         feature = ee.List(feature);
#         M = ee.Number(M);

#         # get the requested network and initialize the created network
#         netData = ee.Feature(feature.get(M.subtract(1)));
#         net = ee.Dictionary();

#         # input slope
#         num = ee.Number(6);
#         start = num.add(1);
#         end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())));
#         net = net.set("inpSlope", ee.List.sequence(start, end).map(lambda ind: getCoefs(netData, ind)))

#         # input offset
#         num = end.add(1)
#         start = num.add(1)
#         end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())));
#         net = net.set("inpOffset", ee.List.sequence(start, end).map(lambda ind: getCoefs(netData, ind)))

#         # hidden layer 1 weight
#         num = end.add(1)
#         start = num.add(1)
#         end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())));
#         net = net.set("h1wt", ee.List.sequence(start, end).map(lambda ind: getCoefs(netData, ind)))

#         # hidden layer 1 bias
#         num = end.add(1)
#         start = num.add(1)
#         end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())));
#         net = net.set("h1bi", ee.List.sequence(start, end).map(lambda ind: getCoefs(netData, ind)))

#         # hidden layer 2 weight
#         num = end.add(1)
#         start = num.add(1)
#         end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())));
#         net = net.set("h2wt", ee.List.sequence(start, end).map(lambda ind: getCoefs(netData, ind)))

#         # hidden layer 2 bias
#         num = end.add(1)
#         start = num.add(1)
#         end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())));
#         net = net.set("h2bi", ee.List.sequence(start, end).map(lambda ind: getCoefs(netData, ind)))

#         # output slope
#         num = end.add(1)
#         start = num.add(1)
#         end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())));
#         net = net.set("outSlope", ee.List.sequence(start, end).map(lambda ind: getCoefs(netData, ind)))

#         # output offset
#         num = end.add(1)
#         start = num.add(1)
#         end = num.add(netData.getNumber(ee.String('tabledata').cat(num.format())));
#         net = net.set("outBias", ee.List.sequence(start, end).map(lambda ind: getCoefs(netData, ind)))

#         return (net)

#     # Parse CSV file with list of networks for a selected variable
#     # This will parse one network for each landclass partition
#     def makeNetVars2(asset, numNets, variableNum):

#         asset = ee.FeatureCollection(asset)
#         numNets = ee.Number(numNets)
#         variableNum = ee.Number(variableNum)

#         # get selected network
#         filtered_features = ee.FeatureCollection(asset.filter(ee.Filter.eq('tabledata3', variableNum))).toList(numNets)

#         # make only first net
#         netList = makeNets2(filtered_features, 1)
#         return netList
#         # return ee.List.sequence(1,numNets).map(lambda netNum: makeNets(filtered_features,netNum))

#     # returns image with single band named network id corresponding given
#     def makeIndexLayer2(image, legend, Network_Ind):

#         image = ee.Image(image)  # partition image
#         legend = ee.FeatureCollection(legend)  # legend to convert partition numbers to networks
#         Network_Ind = ee.FeatureCollection(Network_Ind)  # legend to convert networks to networkIDs

#         # get lists of valid partitions
#         legend_list = legend.toList(legend.size())
#         landcover = legend_list.map(lambda feature: ee.Feature(feature).getNumber('Value'))

#         # get corresponding networkIDs
#         networkIDs = legend_list.map(lambda feature: ee.Feature(feature).get('SL2P Network')) \
#             .map(lambda propertyValue: ee.Feature(ee.FeatureCollection(Network_Ind).first()).toDictionary().getNumber(
#             propertyValue))

#         return image.remap(landcover, networkIDs, 0).rename('networkID')

#     # returns image with single band named networkid corresponding given
#     # input partition image remapped to networkIDs
#     # applies a set of shallow networks to an image based on a provided partition image band
#     def wrapperNNets2(network, partition, netOptions, colOptions, layerName, imageInput):

#         # typecast function parameters
#         network = ee.List(network)
#         partition = ee.Image(partition)
#         netOptions = ee.Dictionary(netOptions)
#         colOptions = ee.Dictionary(colOptions)
#         layerName = ee.String(layerName)
#         imageInput = ee.Image(imageInput)

#         # parse partition  used to identify network to use
#         partition = partition.clip(imageInput.geometry()).select(['partition'])

#         # determine networks based on collection
#         netList = ee.List(network.get(ee.Number(netOptions.get("variable")).subtract(1)))

#         # parse land cover into network index and add to input image
#         imageInput = imageInput.addBands(
#             makeIndexLayer2(partition, colOptions.get("legend"), colOptions.get("Network_Ind")))

#         # define list of input names
#         netIndex = 0;
#         netDict = ee.Dictionary(selectNet2(imageInput, netList, netOptions.get("inputBands"), netIndex));
#         estimate = ee.Image(applyNet2(layerName, netDict))

#         return estimate.addBands(partition).addBands(imageInput.select('networkID'))

#     # input parameters
#     # collection name
#     colName = "COPERNICUS/S2_SR"

#     # product name, one of('Surface_Reflectance','Albedo','FAPAR','FCOVER','LAI','CCC','CWC','DASF')
#     #outputName = "LAI"

#     # date range for inputs
#     # startDate = ee.Date('2020-04-01')
#     # endDate = ee.Date('2020-09-30')
#     #year_start = year
#     #year_end = year
#    # month_start = 4
#    # month_end = 9

#     # geographical bounds of inputs you can specify a JSON geometry (e.g. from earth engine)
#     #table = ee.FeatureCollection('users/ganghong/TilesSampling_Belmanip')
#     # print(table)
#     # Map.addLayer(table)
#     #table = table.map(lambda feat: feat.buffer(-10000))
#     # print(table)
#     #featureNumber = ee.Number(0)
#     #mapBounds = ee.FeatureCollection(ee.Feature(table.toList(table.size()).get(featureNumber))).geometry()
#     # mapBounds =  ee.Geometry.Polygon( \
#     #        [[[-75, 45], \
#     #          [-75, 46], \
#     #          [-74, 46], \
#     #          [-74, 45]]])
#     # other filters - for now only cloud cover
#     maxCloudcover = 60

#     # Output parameters , outputScale and outputOffset not applied to "Surface_Reflectance"
#     outputScale = 1000
#     outputOffset = 0
#     exportFolder = "testSL2Pv3"
#     exportScale = 20
#     exportDatatype = "int"

#     COLLECTION_OPTIONS = {
#         'COPERNICUS/S2_SR': {
#             "name": 'S2',
#             "description": 'Sentinel 2A',
#             "Cloudcover": 'CLOUDY_PIXEL_PERCENTAGE',
#             "Watercover": 'WATER_PERCENTAGE',
#             "sza": 'MEAN_SOLAR_ZENITH_ANGLE',
#             "vza": 'MEAN_INCIDENCE_ZENITH_ANGLE_B8A',
#             "saa": 'MEAN_SOLAR_AZIMUTH_ANGLE',
#             "vaa": 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A',
#             "VIS_OPTIONS": 'VIS_OPTIONS',
#             "Collection_SL2P": ee.FeatureCollection(s2_createFeatureCollection_estimates()),
#             "Collection_SL2Perrors": ee.FeatureCollection(s2_createFeatureCollection_errors()),
#             "sl2pDomain": ee.FeatureCollection(s2_createFeatureCollection_domains()),
#             "Network_Ind": ee.FeatureCollection(s2_createFeatureCollection_Network_Ind()),
#             "partition": ee.ImageCollection(s2_createImageCollection_partition()),
#             "legend": ee.FeatureCollection(s2_createFeatureCollection_legend()),
#             "numVariables": 7
#         },
#         'LANDSAT/LC08/C01/T1_SR': {
#             "name": 'L8',
#             "description": 'LANDSAT 8',
#             "Cloudcover": 'CLOUD_COVER_LAND',
#             "Watercover": 'CLOUD_COVER',
#             "sza": 'SOLAR_ZENITH_ANGLE',
#             "vza": 'SOLAR_ZENITH_ANGLE',
#             "saa": 'SOLAR_AZIMUTH_ANGLE',
#             "vaa": 'SOLAR_AZIMUTH_ANGLE',
#             "VIS_OPTIONS": 'VIS_OPTIONS',
#             "Collection_SL2P": ee.FeatureCollection(l8_createFeatureCollection_estimates()),
#             "Collection_SL2Perrors": ee.FeatureCollection(l8_createFeatureCollection_errors()),
#             "sl2pDomain": ee.FeatureCollection(l8_createFeatureCollection_domains()),
#             "Network_Ind": ee.FeatureCollection(l8_createFeatureCollection_Network_Ind()),
#             "partition": ee.ImageCollection(l8_createImageCollection_partition()),
#             "legend": ee.FeatureCollection(l8_createFeatureCollection_legend()),
#             "numVariables": 7
#         }
#     }
#     VIS_OPTIONS = {
#         "Surface_Reflectance": {
#             "COPERNICUS/S2_SR": {
#                 "Name": 'Surface_Reflectance',
#                 "description": 'Surface_Reflectance',
#                 "inp": ['B4', 'B5', 'B6', 'B7', 'B8A', 'B9', 'B11', 'B12']
#             }
#         },
#         "Albedo": {
#             "COPERNICUS/S2_SR": {
#                 "Name": 'Albedo',
#                 "errorName": 'errorAlbedo',
#                 "maskName": 'maskAlbedo',
#                 "description": 'Black sky albedo',
#                 "variable": 6,
#                 "inputBands": ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
#                 "inputScaling": [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001,
#                                  0.0001],
#                 "outmin": (ee.Image(ee.Array([[0]]))),
#                 "outmax": (ee.Image(ee.Array([[1]])))
#             }
#         },
#         'fAPAR': {
#             "COPERNICUS/S2_SR": {
#                 "Name": 'fAPAR',
#                 "errorName": 'errorfAPAR',
#                 "maskName": 'maskfAPAR',
#                 "description": 'Fraction of absorbed photosynthetically active radiation',
#                 "variable": 2,
#                 "inputBands": ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
#                 "inputScaling": [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001,
#                                  0.0001],
#                 "outmin": (ee.Image(ee.Array([[0]]))),
#                 "outmax": (ee.Image(ee.Array([[1]])))
#             }
#         },
#         'fCOVER': {
#             "COPERNICUS/S2_SR": {
#                 "Name": 'fCOVER',
#                 "errorName": 'errorfCOVER',
#                 "maskName": 'maskfCOVER',
#                 "description": 'Fraction of canopy cover',
#                 "variable": 3,
#                 "inputBands": ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
#                 "inputScaling": [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001,
#                                  0.0001],
#                 "outmin": (ee.Image(ee.Array([[0]]))),
#                 "outmax": (ee.Image(ee.Array([[1]])))
#             }
#         },
#         'LAI': {
#             "COPERNICUS/S2_SR": {
#                 "Name": 'LAI',
#                 "errorName": 'errorLAI',
#                 "maskName": 'maskLAI',
#                 "description": 'Leaf area index',
#                 "variable": 1,
#                 "inputBands": ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
#                 "inputScaling": [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001,
#                                  0.0001],
#                 "outmin": (ee.Image(ee.Array([[0]]))),
#                 "outmax": (ee.Image(ee.Array([[1]])))
#             }
#         },
#         'CCC': {
#             "COPERNICUS/S2_SR": {
#                 "Name": 'CCC',
#                 "errorName": 'errorCCC',
#                 "maskName": 'maskCCC',
#                 "description": 'Canopy chlorophyll content',
#                 "variable": 1,
#                 "inputBands": ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
#                 "inputScaling": [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001,
#                                  0.0001],
#                 "outmin": (ee.Image(ee.Array([[0]]))),
#                 "outmax": (ee.Image(ee.Array([[1000]])))
#             }
#         },
#         'CWC': {
#             "COPERNICUS/S2_SR": {
#                 "Name": 'CWC',
#                 "errorName": 'errorCWC',
#                 "maskName": 'maskCWC',
#                 "description": 'Canopy water content',
#                 "variable": 1,
#                 "inputBands": ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
#                 "inputScaling": [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001,
#                                  0.0001],
#                 "outmin": (ee.Image(ee.Array([[0]]))),
#                 "outmax": (ee.Image(ee.Array([[100]])))
#             }
#         },
#         'DASF': {
#             "COPERNICUS/S2_SR": {
#                 "Name": 'DASF',
#                 "errorName": 'errorDASF',
#                 "maskName": 'maskDASF',
#                 "description": 'Directional area scattering factor',
#                 "variable": 1,
#                 "inputBands": ['cosVZA', 'cosSZA', 'cosRAA', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12'],
#                 "inputScaling": [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001,
#                                  0.0001],
#                 "outmin": (ee.Image(ee.Array([[0]]))),
#                 "outmax": (ee.Image(ee.Array([[1]])))
#             }
#         }
#     }

#     # parse the networks
#     colOptions = COLLECTION_OPTIONS[colName]
#     netOptions = VIS_OPTIONS[outputName][colName]
#     numNets = ee.Number(
#         ee.Feature((COLLECTION_OPTIONS[colName]["Network_Ind"]).first()).propertyNames().remove('Feature Index').remove(
#             'system:index').remove('lon').size())
#     # SL2P1 = makeNetVars2(COLLECTION_OPTIONS[colName]["Collection_SL2P"],numNets,1)
#     SL2P = ee.List.sequence(1, ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]), 1).map(
#         lambda netNum: makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2P"], numNets, netNum))
#     errorsSL2P = ee.List.sequence(1, ee.Number(COLLECTION_OPTIONS[colName]["numVariables"]), 1).map(
#         lambda netNum: makeNetVars(COLLECTION_OPTIONS[colName]["Collection_SL2Perrors"], numNets, netNum))

#     # make products and export

#     # filter collection and add ancillary bands
#     input_collection = ee.ImageCollection(colName) \
#         .filterBounds(mapBounds).filterDate(startDate, endDate).filterMetadata('MGRS_TILE','equals',tilename).filterMetadata(colOptions["Cloudcover"], 'less_than',maxCloudcover) \
#         .limit(5000) \
#         .map(lambda image: image.clip(mapBounds)) \
#         .map(lambda image: s2MaskClear(image)) \
#         .map(lambda image: addS2Geometry(colOptions, image)) \
#         .map(lambda image: addDate(image)) 

#     # print(input_collection.size().getInfo())

#     if outputName == "Surface_Reflectance":
#         export_collection = input_collection;
#     else:
#         # get partition used to select network
#         partition = (COLLECTION_OPTIONS[colName]["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename(
#             'partition')
#         # pre process input imagery and flag invalid inputs
#         input_collection = input_collection.map(lambda image: s2MaskLand(image)) \
#             .map(lambda image: scaleBands(netOptions["inputBands"], netOptions["inputScaling"], image)) \
#             .map(lambda image: invalidInput(COLLECTION_OPTIONS[colName]["sl2pDomain"], netOptions["inputBands"], image))
        
#         # apply networks to produce mapped parameters
#         estimateSL2P = input_collection.map(
#             lambda image: wrapperNNets(SL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "estimate", image))

#         uncertaintySL2P = input_collection.map(
#             lambda image: wrapperNNets(errorsSL2P, partition, netOptions, COLLECTION_OPTIONS[colName], "error", image))
        
#         # scale and offset mapped parameter bands
#         estimateSL2P = estimateSL2P.map(lambda image: image.addBands(
#             image.select("estimate" + outputName).multiply(ee.Image.constant(outputScale)).add(
#                 ee.Image.constant(outputOffset)), overwrite=True))
#         uncertaintySL2P = uncertaintySL2P.map(lambda image: image.addBands(
#             image.select("error" + outputName).multiply(ee.Image.constant(outputScale)).add(
#                 ee.Image.constant(outputOffset)), overwrite=True))
#         # produce final export collection
        
#         #print('Input',input_collection.first().bandNames().getInfo())
#         print(input_collection.first().select('B3').projection().getInfo())
#         #print('Estimate',estimateSL2P.map(lambda image: image.reproject(input_collection.first().select('B3').projection(),None,20).getInfo()))
#         #print('Uncertainty',uncertaintySL2P.getInfo())
#         export_collection = input_collection.combine(estimateSL2P) \
#                                             .combine(uncertaintySL2P) \
#                                             .map(lambda image: image.reproject(image.select('B3').projection(),None,20).clip(mapBounds)) 
                
        

#         #print('export ',export_collection.getInfo())
        
#     return export_collection



**Functions definition**

In [22]:
basemaps = {
'Google Maps': folium.TileLayer(
tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
attr = 'Google',
name = 'Google Maps',
overlay = True,
control = True
),
'Google Satellite': folium.TileLayer(
tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
attr = 'Google',
name = 'Google Satellite',
overlay = True,
control = True
)}

## display layers
def add_ee_layer(self, ee_object, vis_params, name):
 try:
  if isinstance(ee_object, ee.image.Image):
   map_id_dict = ee.Image(ee_object).getMapId(vis_params)
   folium.raster_layers.TileLayer(
   tiles = map_id_dict['tile_fetcher'].url_format,
   attr = 'Google Earth Engine',
   name = name,
   overlay = True,
   control = True
   ).add_to(self)

  elif isinstance(ee_object, ee.imagecollection.ImageCollection):
   ee_object_new = ee_object.mosaic()
   map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
   folium.raster_layers.TileLayer(
   tiles = map_id_dict['tile_fetcher'].url_format,
   attr = 'Google Earth Engine',
   name = name, 
   overlay = True,
   control = True
   ).add_to(self)

  elif isinstance(ee_object, ee.geometry.Geometry):
   folium.GeoJson(
   data = ee_object.getInfo(),
   name = name,
   overlay = True,
   control = True
   ).add_to(self)

  elif isinstance(ee_object,ee.featurecollection.FeatureCollection):
   ee_object_new = ee.Image().paint(ee_object, 0, 2)
   map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
   folium.raster_layers.TileLayer(
   tiles = map_id_dict['tile_fetcher'].url_format,
   attr = 'Google Earth Engine',
   name = name,
   overlay = True,
   control = True
   ).add_to(self)

 except:
  print("Could not display {}".format(name))

#date conversion
def DatetoSystemtime(datelist):
    geestamp=[]
    for date in datelist:       
        date = pd.to_datetime(date, format='%Y-%m-%d')       
       # print (date)
        year_day = pd.Timestamp(year=date.year, month=1, day=1)       
        doy = (date - year_day).days + 1     ## add three digits ###   
        doy='{:>03}'.format(doy) ### three digit format for doy 000
        hour = '12'
        year=date.year
        timestamp = str(year) + doy + hour
        # convert time stamp to gee format
        gee_timestamp = int(pd.to_datetime(timestamp,format='%Y%j%H').value // 10**6)
        #print (gee_timestamp)
        geestamp.append(gee_timestamp)        
    return geestamp    

#remove properties 
def removeProperty(feat): 
    properties = feat.propertyNames()
    selectProperties = properties.filter(ee.Filter.eq('item', 'MGRS_TILE')) 
    return feat.select(selectProperties)

#featurecollection intersection
def featurecollection_interset(tile, resultlist):
    resultlist = ee.List(resultlist)
    tile = ee.Feature(tile)
    def polygonintersect(zone):    
        zone = ee.Feature(zone)   
        intersection = tile.intersection(zone, ee.ErrorMargin(1));
        return intersection.set({'ezone': zone.get('EZMAP_ZONE')}).set({'Intersect': intersection.area().divide(1000 * 1000)})
    intersection = zoneList.map(polygonintersect)
    sortedintersect=ee.FeatureCollection(intersection).sort('Intersect', False)
    return resultlist.add(sortedintersect.first())

#land cover frequency vector based
def calculateFreq(tile,list):
    list=ee.List(list)
    frequency = NALCMS_forest.mosaic().reduceRegion(
       reducer=ee.Reducer.frequencyHistogram(),
       maxPixels=2e9,
       scale=500,
       geometry=tile.geometry()
      )
    tempobj= frequency.get("landcover")
    okeys = ee.Dictionary(tempobj).keys()
    ovals = ee.Dictionary(tempobj).values()
    minmax = ee.Dictionary(ovals.reduce(ee.Reducer.minMax()))
    maxval = ee.Number(minmax.get('max'))
    maxind = ovals.indexOf(maxval)
    maxkey = okeys.get(maxind)
    tile= tile.set({'landcover': maxkey})
    return list.add(tile)

#feature to dataframe
def fc2df(fc):
    # Convert a FeatureCollection into a pandas DataFrame
    # Features is a list of dict with the output
    features = fc.getInfo()['features']
    dictarr = []
    for f in features:
        # Store all attributes in a dict
        attr = f['properties']
        # and treat geometry separately
        #attr['geometry'] = f['geometry']  # GeoJSON Feature!
        # attr['geometrytype'] = f['geometry']['type']
        dictarr.append(attr)
        df = GeoDataFrame(dictarr)
        # Convert GeoJSON features to shape
        #df['geometry'] = map(lambda s: shape(s), df.geometry)
    return df

#add date to a feature
def datelist(img):
    imgdate=ee.Date(img.get("system:time_start"))
    return ee.Feature(None, {'date': imgdate.format('YYYY-MM-dd')})
 
#bit wsie operation  
def bitwiseExtract(value, fromBit, toBit):
    maskSize = ee.Number(1).add(toBit).subtract(fromBit)
    mask = ee.Number(1).leftShift(maskSize).subtract(1)
    return value.rightShift(fromBit).bitwiseAnd(mask)

#modis product bit mask
def bitmask(image):
    qa = image.select('FparLai_QC')
    good = bitwiseExtract(qa, 0,0) # returns 0 for good quality
    return image.updateMask(ee.Image(good).eq(0))  ## needs to be inverted to mask pixels with bad quality, 0 means good quality , pass the mask  

## function for calculate raster dominant 
def dominantCal(raster, coarseProj, coarseScale, targetscale, minVal, maxVal, steps):
    ##calcualte stats from reduceResolution
    #stats = raster.reproject(raster.projection(), None, targetscale).reduceResolution(
    stats = raster.reproject(coarseProj, None, targetscale).reduceResolution(
       reducer= ee.Reducer.fixedHistogram(minVal, maxVal, steps),
       maxPixels= 65535,
       bestEffort= True
    ).reproject(
      crs=coarseProj,
      scale= coarseScale##MODISscale
    )

    array=stats.toArray()
    ## slice to land cover
    landcover=array.arraySlice(1,0,1)
    ##slice to percentage
    percentage=array.arraySlice(1,1,2)
    ##sort percentage
    sortedper = array.arraySort(percentage);
    ## get values
    length = sortedper.arrayLength(0)
    values = sortedper.arraySlice(0, length.subtract(1), length);
    ##convert back to an image  
    maxpercentage = values.arrayProject([1]).arrayFlatten([['landcover', 'percentage']])  ### be careful 'landcover' will be used in the function imgTonumpy(sensor, band_arrs):/ band_dominantLC = band_arrs.get('landcover')
    dominantlc=maxpercentage.select('landcover').ceil()
    return maxpercentage.select('landcover').ceil()

## function for calculate raster dominant 
def dominantCalEZ(raster, coarseProj, coarseScale, targetscale, minVal, maxVal, steps):
    ##calcualte stats from reduceResolution
    #stats = raster.reproject(raster.projection(), None, targetscale).reduceResolution(
    stats = raster.reproject(coarseProj, None, targetscale).reduceResolution(
       reducer= ee.Reducer.fixedHistogram(minVal, maxVal, steps),
       maxPixels= 65535,
       bestEffort= True
    ).reproject(
      crs=coarseProj,
      scale= coarseScale##MODISscale
    )

    array=stats.toArray()
    ## slice to land cover
    ##landcover=array.arraySlice(1,0,1)
    ez=array.arraySlice(1,0,1)
    ##slice to percentage
    percentage=array.arraySlice(1,1,2)
    ##sort percentage
    sortedper = array.arraySort(percentage);
    ## get values
    length = sortedper.arrayLength(0)
    values = sortedper.arraySlice(0, length.subtract(1), length);
    ##convert back to an image  
    maxpercentage = values.arrayProject([1]).arrayFlatten([['ez', 'percentage']]) ### be careful 'ez' will be used in the function imgTonumpy(sensor, band_arrs):/ band_dominantEZ = band_arrs.get('ez')
    dominantlc=maxpercentage.select('ez').ceil()
    return maxpercentage.select('ez').ceil()

## function for add dominat lc and ez
def addDZ(dominantEZ,dominantLC):
    def wrap(img):
        ez=ee.Image(dominantEZ).clip(ee.Image(img).geometry())
        lc=ee.Image(dominantLC).clip(ee.Image(img).geometry())
        return img.addBands(ee.Image(ez).rename('dominantEZ')).addBands(ee.Image(lc).rename('dominantLC'))
        #return img.addBands(dominantEZ.rename('dominantEZ')).addBands(dominantLC.rename('dominantLC'))
    return wrap

 ## Extract the ID of each image 
def  getidList(item):
     return ee.Image(item).id()
 
 ## matched date between sensors   
def datematched(halftempRes,sensorname, colCoarse):
    def wrap(img):
            dateImg=img.get('date')
            startdate= ee.Date(dateImg).advance(-halftempRes,'day')  
            enddate= ee.Date(dateImg).advance(halftempRes,'day')
            #size = colCoarse.filterDate(startdate, enddate).size()  ## dataMODIS is a default reference sensor    
            colCoarse2 = colCoarse.filterDate(startdate, enddate)  ## dataMODIS is a default reference sensor
            size=colCoarse2.size()
            return img.set('matched'+sensorname,ee.Algorithms.If(size.eq(0),'none', colCoarse2.toList(size)))  ## change matcheddata to matcheddata plus sensor
    return wrap

## mask for CLGS data  CLGS_flag
def validCGLSMask(CGLS_flag):
    def wrap(image):
        qa = image.select(CGLS_flag)   ## CGLS has different bands for each parameter, like b3 for LAI, b2 for FAPAR
        land = qa.bitwiseAnd(1 << 0)   ## bit 0 equal to 1, land 
        #return image.unmask().updateMask(land)  ##bit 0 equal to 1 means land ; Goolge Earth Engine masked no-data already
        return image.updateMask(land)  ##bit 0 equal to 1 means land ; Goolge Earth Engine masked no-data already
    return wrap

## mask for MODIS data
def validMODISMask(image):
    qa = image.select('FparLai_QC')
    qaBitsGood = qa.bitwiseAnd(1 << 0) ##bit 0, 0 mean good , 1 bad pixel
    validmask=qaBitsGood.eq(0) 
    return image.updateMask(validmask)


In [19]:
## another solution for converting ee image to array 
##export array
def LatLonImg(img, result):
    img = img.addBands(ee.Image.pixelLonLat()) 
    img = img.reduceRegion(reducer=ee.Reducer.toList(),\
                                        geometry=aoi,\
                                        maxPixels=1e15,\
                                        scale=10);
 
    data = np.array((ee.Array(img.get(result)).getInfo()))
    lats = np.array((ee.Array(img.get("latitude")).getInfo()))
    lons = np.array((ee.Array(img.get("longitude")).getInfo()))
    print (len(lats))
    # get the unique coordinates
    uniqueLats = np.unique(lats)
    uniqueLons = np.unique(lons)
    print (uniqueLats)
 
    # get number of columns and rows from coordinates
    ncols = len(uniqueLons)
    nrows = len(uniqueLats)
    print (ncols)
    print (nrows)
 
    # determine pixelsizes
    ys = uniqueLats[1] - uniqueLats[0]
    xs = uniqueLons[1] - uniqueLons[0]
 
    # create an array with dimensions of image
    arr = np.zeros([nrows, ncols], np.float32) #-9999
 
    # fill the array with values
    counter =0
    for y in range(0,len(arr),1):
        for x in range(0,len(arr[0]),1):
            if lats[counter] == uniqueLats[y] and lons[counter] == uniqueLons[x] and counter < len(lats)-1:
                counter+=1
                arr[len(uniqueLats)-1-y,x] = data[counter] # we start from lower left corner
    return arr

In [23]:
## convert image to numpy array
def imgTonumpy(parameter, sensor, band_arrs):
    
    band1= parameter+'_mean'  ### example: LAI_mean
    band2= parameter+'_stdDev'### example: LAI_stdDev
    band3= parameter+'_count'### example: LAI_count
    band4= parameter+'error_mean'### example: LAIerror_mean
    band5= parameter+'error_count'### example: LAIerror_count    
     # # Get individual band arrays.
    #print( band_arrs.getInfo())
    band_LAImean = band_arrs.get(band1)
    band_LAIstdDev = band_arrs.get(band2)
    band_LAIcount = band_arrs.get(band3)
    band_LAIerrormean = band_arrs.get(band4)
    band_LAIerrorcount = band_arrs.get(band5)   
    #print( band_LAIerrorcount.getInfo())
   
    # # Transfer the arrays from server to client and cast as np array.
    np_LAImean = np.array(band_LAImean.getInfo())
    np_LAIstdDev = np.array(band_LAIstdDev.getInfo())
    np_LAIcount = np.array(band_LAIcount.getInfo())
    np_LAIerrormean = np.array(band_LAIerrormean.getInfo())
    np_LAIerrorcount = np.array(band_LAIerrorcount.getInfo())    
   # print(np_LAImean.shape)
   # print(np_dominantEZ.shape)
     # # Expand the dimensions of the images so they can be concatenated into 3-D.
    np_LAImean = np.expand_dims(np_LAImean, 2)
    np_LAIstdDev = np.expand_dims(np_LAIstdDev, 2)
    np_LAIcount = np.expand_dims(np_LAIcount, 2)
    np_LAIerrormean = np.expand_dims(np_LAIerrormean, 2)
    np_LAIerrorcount = np.expand_dims(np_LAIerrorcount, 2)
    if sensor=='S2':
        band_dominantEZ = band_arrs.get('ez') ### 'ez' comes from function def dominantCalEZ(raster, coarseProj, coarseScale, targetscale, minVal, maxVal, steps):/maxpercentage = values.arrayProject([1]).arrayFlatten([['ez', 'percentage']])
        band_dominantLC = band_arrs.get('landcover')  ### 'landcover' comes from function def dominantCal(raster, coarseProj, coarseScale, targetscale, minVal, maxVal, steps):/maxpercentage = values.arrayProject([1]).arrayFlatten([['landcover', 'percentage']])
        band_water=band_arrs.get('waterCover')
        #print(band_dominantLC.getInfo())
        np_dominantEZ = np.array(band_dominantEZ.getInfo())
        np_dominantLC = np.array(band_dominantLC.getInfo())
        np_water = np.array(band_water.getInfo())
       # print(np_dominantLC.shape)
        np_dominantEZ = np.expand_dims(np_dominantEZ, 2)
        np_dominantLC = np.expand_dims(np_dominantLC, 2)
        np_water = np.expand_dims(np_water, 2)
        img_stack = np.concatenate((np_LAImean, np_LAIstdDev, np_LAIcount, np_LAIerrormean, np_LAIerrorcount,np_dominantEZ, np_dominantLC,np_water), 2)        
    else: 
        img_stack = np.concatenate((np_LAImean, np_LAIstdDev, np_LAIcount, np_LAIerrormean, np_LAIerrorcount), 2)
#     print(np_dominantEZ.shape)
        # # Stack the individual bands to make a 3-D array.
    #img_stack = np.concatenate((np_LAImean, np_LAIstdDev, np_LAIcount, np_LAIerrormean, np_LAIerrorcount), 2)
   # img_stack = np.dstack((np_LAImean, np_LAIstdDev, np_LAIcount, np_LAIerrormean, np_LAIerrorcount,np_dominantEZ, np_dominantLC))
   # img_stack = np.dstack((np_LAImean, np_LAIstdDev, np_LAIcount, np_LAIerrormean, np_LAIerrorcount))
    #img_stack = np.concatenate((np_LAImean, np_LAIstdDev, np_LAIcount, np_LAIerrormean, np_LAIerrorcount,np_dominantEZ, np_dominantLC), 2)        
    return img_stack

In [22]:
def imgTonumpy2(sensor, band_arrs):
     # # Get individual band arrays.
    #print( band_arrs.getInfo())
    band_LAImean = band_arrs.aggregate_array('LAI_mean')
    band_LAIstdDev = band_arrs.aggregate_array('LAI_stdDev')
    band_LAIcount = band_arrs.aggregate_array('LAI_count')
    band_LAIerrormean = band_arrs.aggregate_array('LAIerror_mean')
    band_LAIerrorcount = band_arrs.aggregate_array('LAIerror_count')   
    #print( band_LAIerrorcount.getInfo())
   
    # # Transfer the arrays from server to client and cast as np array.
    np_LAImean = np.array(band_LAImean.getInfo())
    np_LAIstdDev = np.array(band_LAIstdDev.getInfo())
    np_LAIcount = np.array(band_LAIcount.getInfo())
    np_LAIerrormean = np.array(band_LAIerrormean.getInfo())
    np_LAIerrorcount = np.array(band_LAIerrorcount.getInfo())    
   # print(np_LAImean.shape)
   # print(np_dominantEZ.shape)
     # # Expand the dimensions of the images so they can be concatenated into 3-D.
    np_LAImean = np.expand_dims(np_LAImean, 1)
    np_LAIstdDev = np.expand_dims(np_LAIstdDev, 1)
    np_LAIcount = np.expand_dims(np_LAIcount, 1)
    np_LAIerrormean = np.expand_dims(np_LAIerrormean, 1)
    np_LAIerrorcount = np.expand_dims(np_LAIerrorcount, 1)
    if sensor=='S2':
        band_dominantEZ = band_arrs.aggregate_array('ez') ### 'ez' comes from function def dominantCalEZ(raster, coarseProj, coarseScale, targetscale, minVal, maxVal, steps):/maxpercentage = values.arrayProject([1]).arrayFlatten([['ez', 'percentage']])
        band_dominantLC = band_arrs.aggregate_array('landcover')  ### 'landcover' comes from function def dominantCal(raster, coarseProj, coarseScale, targetscale, minVal, maxVal, steps):/maxpercentage = values.arrayProject([1]).arrayFlatten([['landcover', 'percentage']])
        #print(band_dominantLC.getInfo())
        np_dominantEZ = np.array(band_dominantEZ.getInfo())
        np_dominantLC = np.array(band_dominantLC.getInfo())
       # print(np_dominantLC.shape)
        np_dominantEZ = np.expand_dims(np_dominantEZ, 1)
        np_dominantLC = np.expand_dims(np_dominantLC, 1)
        img_stack = np.concatenate((np_LAImean, np_LAIstdDev, np_LAIcount, np_LAIerrormean, np_LAIerrorcount,np_dominantEZ, np_dominantLC), 1)        
    else: 
        img_stack = np.concatenate((np_LAImean, np_LAIstdDev, np_LAIcount, np_LAIerrormean, np_LAIerrorcount), 1)
#     print(np_dominantEZ.shape)
        # # Stack the individual bands to make a 3-D array.
    #img_stack = np.concatenate((np_LAImean, np_LAIstdDev, np_LAIcount, np_LAIerrormean, np_LAIerrorcount), 2)
   # img_stack = np.dstack((np_LAImean, np_LAIstdDev, np_LAIcount, np_LAIerrormean, np_LAIerrorcount,np_dominantEZ, np_dominantLC))
   # img_stack = np.dstack((np_LAImean, np_LAIstdDev, np_LAIcount, np_LAIerrormean, np_LAIerrorcount))
    #img_stack = np.concatenate((np_LAImean, np_LAIstdDev, np_LAIcount, np_LAIerrormean, np_LAIerrorcount,np_dominantEZ, np_dominantLC), 2)        
    return img_stack

In [24]:
## convert GEE image to numpy array
def imageToarray(parameter, aoi, coarseS2, coarseCGLS, coarseMODIS):

    imgList_S2 = coarseS2.toList(coarseS2.size())  # convert collection to list
    print('size of imageList_S2:  ', imgList_S2.size().getInfo())

    sensorList = []  # sensor list: ouput of this functin:  element is dictionary, S2: array, MODIS: array/'none', CGLS: array/'none'
    images_dict = defaultdict(list)
    
    band1= parameter+'_mean'  ### example: LAI_mean
    band2= parameter+'_stdDev'### example: LAI_stdDev
    band3= parameter+'_count'### example: LAI_count
    band4= parameter+'error_mean'### example: LAIerror_mean
    band5= parameter+'error_count'### example: LAIerror_count    

    for i in range(coarseS2.size().getInfo()):
        ## get image from image list
        image_S2 = ee.Image(imgList_S2.get(i)).select([band1, band2, band3,band4, band5,'dominantEZ','dominantLC','waterCover'])
        #print (image_S2.projection().getInfo())
        #print (image_S2.getInfo()['bands'])
       # print (image_S2.getInfo()['bands'][0]['dimensions'])
        #print (image_S2.getInfo()['bands'][0])
        #print (image_S2.propertyNames().getInfo())
        # Get 2-d pixel array for AOI - returns feature with 2-D pixel array as property per band.
        band_S2_arrs = image_S2.sampleRectangle(region=aoi, defaultValue=255)
        #band_S2_arrs = image_S2.sample(region=aoi,  geometries=True)
       # band_S2_arrs = image_S2.select('LAI_mean').sampleRectangle()
        #band_S2_arrs = image_S2.sample(region=aoi, scale=500)
        #print(band_S2_arrs.getInfo())
     #   print(band_S2_arrs.get('LAI_mean').getInfo())
        S2_img=imgTonumpy(parameter,'S2',band_S2_arrs)
# #         # S2_img = geemap.ee_to_numpy(image_S2, region=aoi)
        print('S2:',S2_img.shape)
        images_dict['S2'].append(S2_img)
        images_dict['S2_size']=S2_img.shape
        images_dict['S2_date']=image_S2.get('date').getInfo()
#         images_dict['S2']='none'
#         images_dict['S2_size']=0

        ### extract images from MODIS
        MODISlist_S2 = image_S2.get('matchedMODIS')  ## get the property 'matchedMODIS' of  the image 
        if (MODISlist_S2.getInfo() != 'none'):
            MODISlist_S2 = ee.List(MODISlist_S2).map(getidList)  ## get the ids of images in the property 'matchedData'
        ### get the corresponding coarse MODIS
        ## use the sytem index to locate coarse MODIS since the matching was occured before the coarse resolution, the image in the matched property is original images
            coarseMODIS_S2 = coarseMODIS.filter(ee.Filter.inList('system:index', MODISlist_S2))
            ### convert coarse MODIS to List
            imgList_MODIS = coarseMODIS_S2.toList(coarseMODIS_S2.size())  # convert collection to list
            
            for j in range(coarseMODIS_S2.size().getInfo()):
                # get image from MODIS list
                image_MODIS = ee.Image(imgList_MODIS.get(j)).select([band1, band2, band3,band4, band5])##,'dominantEZ','dominantLC'])##.unmask()
                #image_MODIS2=image_MODIS.addBands(image_S2.select('LAI_mean').rename('S2_LAI_mean'))
                #image_MODIS2=image_MODIS.addBands(image_MODIS.select('LAI_mean').add(image_S2.select('LAI_mean')),overwrite=True)
               
                #image_MODIS=image_MODIS2
                # Get 2-d pixel array for AOI - returns feature with 2-D pixel array as property per band.
                #print (image_MODIS.getInfo()['bands'])
                band_arrs_MODIS = image_MODIS.sampleRectangle(region=aoi, defaultValue=255)
                #band_arrs_MODIS = image_MODIS.sample(region=aoi,  geometries=True)
                #print (band_arrs_MODIS.getInfo())              
                MODIS_img=imgTonumpy(parameter,'MODIS',band_arrs_MODIS)     
               # MODIS_img = geemap.ee_to_numpy(image_MODIS, region=aoi)
                print('MODIS',MODIS_img.shape)
                images_dict['MODIS'].append(MODIS_img)
                #images_dict['MODIS'] = 'MODIS'
                
            images_dict['MODIS_size'] = coarseMODIS_S2.size().getInfo()
        else:
            images_dict['MODIS'] = 'none'
            images_dict['MODIS_size']=0
            

        ### extract images from CGLS
        CGLSlist_S2 = image_S2.get('matchedCGLS')  ## get the property 'matchedData' of  first image of the list
        if (CGLSlist_S2.getInfo() != 'none'):
            CGLSlist_S2 = ee.List(CGLSlist_S2).map(getidList)  ## get the ids of images in the property 'matchedData'
        ### get the corresponding coarse MODIS
        ## use the sytem index to locate coarse MODIS since the matching was occured before the coarse resolution, the image in the matched property is original images
            coarseCGLS_S2 = coarseCGLS.filter(ee.Filter.inList('system:index', CGLSlist_S2))
            ### convert coarse MODIS to List
            imgList_CGLS = coarseCGLS_S2.toList(coarseCGLS_S2.size())  # convert collection to list

            for k in range(coarseCGLS_S2.size().getInfo()):
                image_CGLS = ee.Image(imgList_CGLS.get(k)).select([band1, band2, band3,band4, band5])##,'dominantEZ','dominantLC'])##.unmask()
                # Get 2-d pixel array for AOI - returns feature with 2-D pixel array as property per band.
                band_arrs_CGLS = image_CGLS.sampleRectangle(region=aoi, defaultValue=255)
                #band_arrs_CGLS = image_CGLS.sample(region=aoi,  geometries=True)
                #print (band_arrs_CGLS.getInfo())               
                CGLS_img=imgTonumpy(parameter,'CGLS',band_arrs_CGLS)   
                #CGLS_img = geemap.ee_to_numpy(image_CGLS, region=aoi)
                print('CGLS',CGLS_img.shape)
                ### add image to dictionary
                images_dict['CGLS'].append(CGLS_img)
                #images_dict['CGLS'] = 'CGLS'
                
            images_dict['CGLS_size'] = coarseCGLS_S2.size().getInfo()
        else:
            images_dict['CGLS'] = 'none'
            images_dict['CGLS_size']=0

        ### add dictionary to list
        sensorList.append(images_dict.copy())
        images_dict.clear() ## clear values of dictionary

    return sensorList

In [25]:
##the function to get the corase resoltuion image collection with matched sensor information 
## startDate, endDate: dates to start and end
##mapBounds: tile boundary 
## tilename: MGRS tile used to filter data, used in S2 only
##parameter: LAI, fAPAR,fCOVER,make sure correponding to name definition leaf-tool box
##tempRes: temporal resolution, like 5 days, 7 days
##coarsescale: scale used for getting the coarse resolution
## MODIS_paraBand, MODIS_error: names of MODIS parameter and uncertainty bands
## S2_paraBand, S2_error: names of S2 parameter and uncertianty bands
## CGLS_parameter: name of parameter Collection, LAI_CCRS in the import data section
## CGLS_paraBand, CGLS_error, CGLS_flag: names of CGLS parameter , uncertianty and quality flag bands
def applySensorComp(year_start,year_end,month_start,month_end, mapBounds, tilename, parameter, tempRes, dataS2_col, coarsescale, MODIS_paraBand, MODIS_error, S2_paraBand, S2_error, CGLS_parameter, CGLS_paraBand, CGLS_error, CGLS_flag):
    
    print('***In applySensorComp****')
    
    para_error=parameter+'error' ## 'LAIerror'
     ### Subsetting image collections,  property setting, flag mask updating
    ##update mask and set a new property date
    dataMODIS =ee.ImageCollection('MODIS/006/MCD15A3H')\
                      .filterBounds(mapBounds)\
                      .filter(ee.Filter.calendarRange(year_start,year_end,'year')).filter(ee.Filter.calendarRange(month_start,month_end,'month'))\
                      .map(lambda img:img.clip(mapBounds))\
                      .map(validMODISMask)\
                      .map(lambda img:img.select([MODIS_paraBand, MODIS_error],[parameter, para_error]).set('date', ee.Date(img.get('system:time_start')).format("yyyy-MM-dd")))   #['Lai','LaiStdDev'],['LAI','LAIun']
    #print ("original MODIS size:",dataMODIS.first().getInfo()['bands'][0]['dimensions'])
    print ("original MODIS band 0:",dataMODIS.first().getInfo()['bands'][0])
    print ("original MODIS scale:",dataMODIS.first().projection().nominalScale().getInfo())
#     dataCGLS=CGLS_parameter.filterBounds(mapBounds)\
#                             .filter(ee.Filter.calendarRange(year_start,year_end,'year')).filter(ee.Filter.calendarRange(month_start,month_end,'month'))\
#                             .map(lambda img:img.clip(mapBounds))\
#                             .map(validCGLSMask(CGLS_flag))\
#                             .map(lambda img:img.select([CGLS_paraBand,CGLS_error],[parameter, para_error]).set('date', ee.Date(img.get('system:time_start')).format("yyyy-MM-dd"))) 
## change the filter to the specific file name instead of tile boudary which cause other adjacent tiles selected as well.
    dataCGLS=CGLS_parameter.filter(ee.Filter.stringContains('id_no', tilename))\
                           .filter(ee.Filter.calendarRange(year_start,year_end,'year')).filter(ee.Filter.calendarRange(month_start,month_end,'month'))\
                           .map(validCGLSMask(CGLS_flag))\
                           .map(lambda img:img.select([CGLS_paraBand,CGLS_error],[parameter, para_error]).set('date', ee.Date(img.get('system:time_start')).format("yyyy-MM-dd"))) 
    ##.map(lambda img:img.clip(mapBounds))\
    #print ("original dataCGLS size:",dataCGLS.first().getInfo()['bands'][0]['dimensions'])
    print ("original dataCGLS band 0:",dataCGLS.first().getInfo()['bands'][0])
    print ("original dataCGLS scale:",dataCGLS.first().projection().nominalScale().getInfo())
   # print ("original S2 size:",dataS2.first().projection().nominalScale().getInfo())
   
# #     dataS2=Leaf_tools(mapBounds,starDate,endDate,tilename,parameter)\
#                      .map(lambda img:img.updateMask(img.select('QC').eq(0)))\
#                      .map(lambda img:img.select([S2_paraBand,S2_error],[parameter, para_error]).set('date', ee.Date(img.get('system:time_start')).format("yyyy-MM-dd")))

    dataS2=dataS2_col.map(lambda img:img.updateMask(img.select('QC').eq(0)))\
                     .map(lambda img:img.select([S2_paraBand,S2_error],[parameter, para_error]).set('date', ee.Date(img.get('system:time_start')).format("yyyy-MM-dd")))\
                     .map(lambda img:img.clip(mapBounds))

    
    ##  .map(lambda img:img.clip(mapBounds))\  ### no clip mapbounds
   # print ('****************done leaf *****************')
  
    print ("original S2 size:",dataS2.first().getInfo()['bands'][0]['dimensions'])
    print ("original S2 band 0:",dataS2.first().getInfo()['bands'][0])
    print ("original S2 scale:",dataS2.first().projection().nominalScale().getInfo())
    
    ### date matching    
    halftempRes=math.ceil(tempRes/2)
    ##print (halftempRes)
   # dataS2matched= dataS2.map(datematched(halftempRes,'MODIS',dataMODIS))    
   # dataCGLSmatched= dataCGLS.map(datematched(halftempRes,'MODIS',dataMODIS))
    dataS2matched= dataS2.map(datematched(halftempRes,'MODIS',dataMODIS)).map(datematched(halftempRes,'CGLS',dataCGLS))
    
    ### coarse resolution reducers  
    reducers = ee.Reducer.mean().combine(
      reducer2= ee.Reducer.stdDev(),
      sharedInputs= True
    ).combine(
      reducer2= ee.Reducer.count(),
      sharedInputs= True
    )
   # coarsecrs=coarsecollection.first().projection()
    coarsecrs=dataS2.first().projection()
    #coarsetrans=coarsecollection.first().projection().transform()
    ##coarse CGLS-product1
    ##coarseCGLS=dataCGLS.map(lambda img:img.reproject(dataCGLS.first().projection(), None, 300).reduceResolution(
    coarseCGLS=dataCGLS.map(lambda img:img.reproject(dataCGLS.first().projection(), None, 300).reduceResolution(
               reducer= reducers,
               maxPixels= 65535,
               bestEffort= True
               ).reproject(
                   crs=coarsecrs,  ##crs= coarsecrs  crs=proj_MODIS
                   scale= coarsescale
                 )).map(lambda img:img.clip(mapBounds))
    ##coarse S2-product2
    ##print (dataS2matched.first().projection().getInfo())
    ### questions here: two options about reprojection setting: one: dataMODIS.first().projection() need not set boundary buffer too big (-40000) 
    ## the other: dataMODIS.first().projection(), None, 20, need not set boundary buffer as -40000 
    #dataS2matched.first().projection(), None, 20, not working, too large
    #coarseS2=dataS2matched.map(lambda img:img.reproject(dataMODIS.first().projection()).reduceResolution(    
    ###???clip S2 accoridning to original S2 boundary
#     dataset = ee.ImageCollection('COPERNICUS/S2_SR')\
#                   .filterDate('2020-01-01', '2020-01-30')\
#                   .filterBounds(mapBounds)
#     S2projection=dataset.first().select('B7').projection()
#     print (S2projection.getInfo())
#     print (S2projection.nominalScale().getInfo())
#     reprojectedS2=dataS2matched.map(lambda img:img.clip(img.geometry().buffer(-10000)).reproject(S2projection, None, 20))
#     print (reprojectedS2.first().projection().getInfo())
#     coarseS2=reprojectedS2.map(lambda img:img.reduceResolution(
#     coarseS2=dataS2matched.map(lambda img:img.clip(img.geometry().buffer(-10000)).reproject(S2projection, None, 20).reduceResolution(
##coarseS2=dataS2matched.map(lambda img:img.clip(img.geometry().buffer(-10000)).reproject(dataS2matched.first().projection(), None, 20).reduceResolution( works
    #coarseS2=dataS2matched.map(lambda img:img.clip(img.geometry().intersection(**{'right': mapBounds, 'maxError': 1})).reduceResolution(       
    #img.geometry().buffer(-30000)
    coarseS2=dataS2matched.map(lambda img:img.reproject(dataS2matched.first().projection(), None, 20).reduceResolution(       
               reducer= reducers,
               maxPixels=65535, 
               bestEffort= True
               ).reproject(
                   crs=coarsecrs,  ##crs= coarsecrs crs=proj_MODIS
                   scale= coarsescale   #### use csr.transform instead of scale scale= coarsescale   crsTransform= [500.00000000022476, 0, -20015109.353988, 0, 499.9999999997751, 10007554.676994] crsTransform= coarsecrs.transform()
                 )).map(lambda img:img.clip(mapBounds))
    #### coarse MODIS product
    ##coarseMODIS=dataMODIS.map(lambda img:img.reproject(dataMODIS.first().projection(), None, 500).reduceResolution(
    coarseMODIS=dataMODIS.map(lambda img:img.reproject(dataMODIS.first().projection(), None, 500).reduceResolution(
               reducer= reducers,
               maxPixels= 65535,
               bestEffort= True
               ).reproject(
                    crs=coarsecrs,  ##crs= coarsecrs crs=proj_MODIS
                   scale= coarsescale
                 )).map(lambda img:img.clip(mapBounds))
    
    print ("coarse S2 projection:",coarseS2.first().projection().getInfo())
    print ("coarse S2 scale:",coarseS2.first().projection().nominalScale().getInfo())
    ### dominant land cover
    tilelandcover=NALCMS_forest.filterBounds(mapBounds).mosaic().clip(mapBounds)  
    dominantLC=dominantCal(tilelandcover, coarsecrs,  coarsescale, 90, 0, 19, 19) ##(0, 19, 19)
    print (dominantLC.getInfo(), 'Land cover')
    ## convert ezone ploygon to raster
    ezraster=ee.FeatureCollection(ezone).reduceToImage(['EZMAP_CODE'],ee.Reducer.first( ))
    tileez=ezraster.clip(mapBounds)
     ### dominant ezone
    dominantEZ=dominantCalEZ(tileez, coarsecrs, coarsescale, 200, 0, 99, 99) ##(0, 99, 99)
    print (dominantEZ.getInfo(),'EZ')
    
    coarseLC=dominantLC.clip(mapBounds).reduceResolution(
               reducer= ee.Reducer.mode(),
               maxPixels= 65535,
               bestEffort= True
               ).reproject(
                   crs=coarsecrs,  ##crs= coarsecrs crs=proj_MODIS
                   scale= coarsescale)               
 
    coarseEZ=dominantEZ.clip(mapBounds).reduceResolution(
               reducer= ee.Reducer.mode(),
               maxPixels= 65535,
               bestEffort= True
               ).reproject(
                   crs=coarsecrs,  ##crs= coarsecrs crs=proj_MODIS
                   scale= coarsescale)
    ### water cover to coarse resoluton for percentage in coarse resolution pixel
    waterCover=NALCMS_water.clip(mapBounds).reproject(NALCMS_water.projection(), None, 30).reduceResolution(
                                          reducer= ee.Reducer.mean(),
                                          bestEffort= True,
                                          maxPixels= 65535
                                        ).multiply(ee.Image.pixelArea()).divide(coarsescale*coarsescale).reproject(
                                  crs=coarsecrs,
                                  scale=coarsescale
                                )
                 
    print ("coarse LC band 0:",coarseLC.getInfo()['bands'][0])
   # print ("coarse LC :",coarseLC.getInfo())
    print ("coarse EZ band 0:",coarseEZ.getInfo()['bands'][0])
    ## add dominant land cover,ezone
    #coarseCGLSwithDominant=coarseCGLS.map(addDZ(coarseEZ,coarseLC))   
     ## add dominant land cover,ezone    
    coarseS2withDominant=coarseS2.map(addDZ(coarseEZ,coarseLC)).map(lambda img:img.addBands(ee.Image(waterCover).rename('waterCover')))
    ## add dominant land cover,ezone
    #coarseMODISwithDominant=coarseMODIS.map(addDZ(coarseEZ,coarseLC))    
    ### return three coarse image collection, the matched MODIS data information include the property matchedData, 
    ###but the images information included is still orignal MODIS images, 
    ##it could be updated through getting system id from original MODIS and use ee.Filter.inList to find the corresponding images in coarse MODIS collection.    
    #return coarseS2withDominant, coarseCGLSwithDominant,coarseMODISwithDominant  
    return coarseS2withDominant, coarseCGLS,coarseMODIS 

In [26]:
### create a collection from GEE asset based on tilename to save the collection size
def createCol(tilename):
#     assetList = ee.List(ee.data.getList({'id':"users/ganghong/S2LAI"}))
#     assetList = ee.List(ee.data.getList({'id':"users/ganghong/S2LAICCRS"}))  ### need update for corresponding parameter
    assetList = ee.List(ee.data.getList({'id':"users/ganghong/S2FAPARCCRS"}))  ### need update for corresponding parameter
    k=0
    for ind in range(0, assetList.length().getInfo()):
        value = ee.Dictionary(assetList.get(ind)) 
        idval = ee.String(value.get('id')).getInfo()
        if (tilename.getInfo() in idval):
            #print (idval)
            if k==0:
                imgCol=ee.ImageCollection(ee.Image(idval))
            else: 
                imgCol=imgCol.merge(ee.Image(idval))
            k=k+1
    print ("S2 size:", imgCol.size().getInfo())
    return imgCol

In [27]:
## output from numpy to csv file
def outputCSV(arraynp_output, tilename, parameter):
    # pair S2-MODIS
    S2MODIS_list=[]
    S2MODIS_STD_list=[]
    S2MODIS_COUNT_list=[]
    S2MODIS_ERROR_list=[]
    S2MODIS_EZ_list=[]
    S2MODIS_LC_list=[]
    S2MODIS_water_list=[]
    S2MODIS_date_list=[]
    MODIS_list=[]
    MODIS_STD_list=[]
    MODIS_COUNT_list=[]
    MODIS_ERROR_list=[]
    # pair S2-CGLS
    S2CGLS_list=[]
    S2CGLS_STD_list=[]
    S2CGLS_COUNT_list=[]
    S2CGLS_ERROR_list=[]
    S2CGLS_EZ_list=[]
    S2CGLS_LC_list=[]
    S2CGLS_water_list=[]
    S2CGLS_date_list=[]
    CGLS_list=[]
    CGLS_STD_list=[]
    CGLS_COUNT_list=[]
    CGLS_ERROR_list=[]

    for i in range(0, len(arraynp_output)-1):
       # print (i)
        S2=np.array(arraynp_output[i]['S2'])
        S2_para=S2[0,:,:,0]
        S2_STD=S2[0,:,:,1]
        S2_COUNT=S2[0,:,:,2]
        S2_ERROR=S2[0,:,:,3]
        S2_EZ=S2[0,:,:,5]
        S2_LC=S2[0,:,:,6]
        S2_water=S2[0,:,:,7]
        S2_date=np.array(arraynp_output[i]['S2_date'])
        if np.array(arraynp_output[i]['MODIS_size'])!=0:
            MODIS=np.array(arraynp_output[i]['MODIS'])
            MODIS_para=MODIS[0,:,:,0]
            MODIS_STD=MODIS[0,:,:,1]
            MODIS_COUNT=MODIS[0,:,:,2]
            MODIS_ERROR=MODIS[0,:,:,3]
            for j in range(0,S2_para.shape[0]):
                for k in range(0,S2_para.shape[1]):
                    S2MODIS_element = S2_para[j][k]
                    MODIS_element=MODIS_para[j][k]
                    if (S2_para[j][k]!=255)&(MODIS_para[j][k]!=255)&(MODIS_ERROR[j][k]!=255):
                        S2MODIS_list.append(S2_para[j][k])
                        S2MODIS_STD_list.append(S2_STD[j][k])
                        S2MODIS_COUNT_list.append(S2_COUNT[j][k])
                        S2MODIS_ERROR_list.append(S2_ERROR[j][k])
                        S2MODIS_EZ_list.append(S2_EZ[j][k])
                        S2MODIS_LC_list.append(S2_LC[j][k])
                        S2MODIS_water_list.append(S2_water[j][k])
                        S2MODIS_date_list.append(S2_date)
                        MODIS_list.append(MODIS_para[j][k])
                        MODIS_STD_list.append(MODIS_STD[j][k])
                        MODIS_ERROR_list.append(MODIS_ERROR[j][k])
        if np.array(arraynp_output[i]['CGLS_size'])!=0:
            CGLS=np.array(arraynp_output[i]['CGLS'])
            CGLS_para=CGLS[0,:,:,0]
            CGLS_STD=CGLS[0,:,:,1]
            CGLS_COUNT=CGLS[0,:,:,2]
            CGLS_ERROR=CGLS[0,:,:,3]
            for m in range(0,S2_para.shape[0]):
                for n in range(0,S2_para.shape[1]):               
                    if (S2_para[m][n]!=255)&(CGLS_para[m][n]!=255.)&(CGLS_ERROR[m][n]!=255.):
                        S2CGLS_list.append(S2_para[m][n])
                        S2CGLS_STD_list.append(S2_STD[m][n])
                        S2CGLS_COUNT_list.append(S2_COUNT[m][n])
                        S2CGLS_ERROR_list.append(S2_ERROR[m][n])
                        S2CGLS_EZ_list.append(S2_EZ[m][n])
                        S2CGLS_LC_list.append(S2_LC[m][n])
                        S2CGLS_water_list.append(S2_water[m][n])
                        S2CGLS_date_list.append(S2_date)
                        CGLS_list.append(CGLS_para[m][n])
                        CGLS_STD_list.append(CGLS_STD[m][n])
                        CGLS_ERROR_list.append(CGLS_ERROR[m][n])            

    S2MODIS_KEY1='S2MODIS_'+parameter
    S2MODIS_KEY2='S2MODIS_'+parameter+'STD'
    S2MODIS_KEY3='S2MODIS_'+parameter+'COUNT'
    S2MODIS_KEY4='S2MODIS_'+parameter+'ERROR'
    S2MODIS_KEY5='S2MODIS_'+'LC'
    S2MODIS_KEY6='S2MODIS_'+'EZ'
    S2MODIS_KEY7='S2MODIS_'+'WaterFraction'
    S2MODIS_KEY8='S2MODIS_'+'DATE'
    MODIS_KEY1='MODIS_'+parameter
    MODIS_KEY2='MODIS_'+parameter+'STD'
    MODIS_KEY3='MODIS_'+parameter+'ERROR'

    S2CGLS_KEY1='S2CGLS_'+parameter
    S2CGLS_KEY2='S2CGLS_'+parameter+'STD'
    S2CGLS_KEY3='S2CGLS_'+parameter+'COUNT'
    S2CGLS_KEY4='S2CGLS_'+parameter+'ERROR'
    S2CGLS_KEY5='S2CGLS_'+'LC'
    S2CGLS_KEY6='S2CGLS_'+'EZ'
    S2CGLS_KEY7='S2CGLS_'+'WaterFraction'
    S2CGLS_KEY8='S2CGLS_'+'DATE'
    CGLS_KEY1='CGLS_'+parameter
    CGLS_KEY2='CGLS_'+parameter+'STD'
    CGLS_KEY3='CGLS_'+parameter+'ERROR'

    S2MODIS_dict={S2MODIS_KEY1:S2MODIS_list,S2MODIS_KEY2:S2MODIS_STD_list,S2MODIS_KEY3:S2MODIS_COUNT_list,S2MODIS_KEY4:S2MODIS_ERROR_list, S2MODIS_KEY5:S2MODIS_LC_list, S2MODIS_KEY6:S2MODIS_EZ_list, 
                  MODIS_KEY1:MODIS_list, MODIS_KEY2:MODIS_STD_list,MODIS_KEY3:MODIS_ERROR_list,S2MODIS_KEY7:S2MODIS_water_list,S2MODIS_KEY8:S2MODIS_date_list}

    S2CGLS_dict={S2CGLS_KEY1:S2CGLS_list,S2CGLS_KEY2:S2CGLS_STD_list,S2CGLS_KEY3:S2CGLS_COUNT_list,S2CGLS_KEY4:S2CGLS_ERROR_list, S2CGLS_KEY5:S2CGLS_LC_list, S2CGLS_KEY6:S2CGLS_EZ_list, 
                  CGLS_KEY1:CGLS_list, CGLS_KEY2:CGLS_STD_list,CGLS_KEY3:CGLS_ERROR_list,S2CGLS_KEY7:S2CGLS_water_list,S2CGLS_KEY8:S2CGLS_date_list}

    df_S2MODIS = pd.DataFrame(S2MODIS_dict)
    df_S2CGLS = pd.DataFrame(S2CGLS_dict)
    
    df_S2MODIS.to_csv(tilename.getInfo()+'_'+parameter+'_S2_MODIS.csv')
    df_S2CGLS.to_csv(tilename.getInfo()+'_'+parameter+'_S2_CGLS.csv')

**one tile test**

In [15]:
#startDate='2020-06-01'
#endDate='2020-08-30'
year_start = 2019
year_end = 2019
month_start = 6
month_end = 6
#tiles_CCRS_buf = tiles_CCRS.map(lambda feat: feat.buffer(-10000))
#mapBounds_buf= ee.FeatureCollection(ee.Feature(tiles_CCRS_buf.toList(tiles_CCRS_buf.size()).get(2))).geometry()

mapBounds= ee.FeatureCollection(ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(3))).geometry()
#print (mapBounds.getInfo())
tilename=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(3)).get('MGRS_TILE')
print('MGRS tilename:',tilename.getInfo())

## create a collection for S2 from GEE asset
S2_imgCol=createCol(tilename)
S2_input=S2_imgCol.filter(ee.Filter.calendarRange(year_start,year_end,'year')).filter(ee.Filter.calendarRange(month_start,month_end,'month')).filterMetadata('MGRS_TILE','equals',tilename)
### run function applySensorComp
coarseS2_output, coarseCGLS_output,coarseMODIS_output=applySensorComp(year_start,year_end, month_start, month_end, mapBounds, tilename, 'LAI', 4, S2_input,  1500, 'Lai','LaiStdDev', 'estimateLAI','errorLAI', LAI_Bel,'b1','b2','b3')
print (coarseS2_output.size().getInfo(), 'coarseS2_output')
print ('coarseS2 band name:',coarseS2_output.first().bandNames().getInfo())  
print (coarseCGLS_output.size().getInfo(), 'coarseCGLS_output')
print (coarseMODIS_output.size().getInfo(), 'coarseMODIS_output')
print (coarseS2_output.first().getInfo()['bands'][0], 'CoarseS2 band 0')
print (coarseCGLS_output.first().getInfo()['bands'][0], 'Coarse CGLS band 0')
print (coarseMODIS_output.first().getInfo()['bands'][0], 'Coarse MODIS band 0')
coarsecrs=coarseS2_output.first().select(0).projection()
coarsescale=coarseS2_output.first().select(0).projection().nominalScale()
print (coarsecrs.getInfo(), 'Coarse S2 projection')
print (coarsescale.getInfo(), 'Coarse S2 scale')

MGRS tilename: 14UPD
S2 size: 35
***In applySensorComp****
original MODIS band 0: {'id': 'LAI', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [552, 245], 'origin': [28870, 8822], 'crs': 'SR-ORG:6974', 'crs_transform': [463.312716528, 0, -20015109.354, 0, -463.312716527, 10007554.677]}
original MODIS scale: 463.3127165275
original dataCGLS band 0: {'id': 'LAI', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [565, 343], 'crs': 'EPSG:4326', 'crs_transform': [0.0029761904762040103, 0, -97.53720238057738, 0, -0.002976190476189799, 53.2425595238156]}
original dataCGLS scale: 331.3080083140298
original S2 size: [9334, 5665]
original S2 band 0: {'id': 'LAI', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -2147483648, 'max': 2147483647}, 'dimensions': [9334, 5665], 'crs': 'EPSG:4326', 'crs_transform': [0.0001796630568239043, 0, -97.53530062550433, 0, -0.0001796630568239043, 53.2404519439116

In [16]:
arraynp_output=imageToarray(mapBounds, coarseS2_output, coarseCGLS_output, coarseMODIS_output)
print (len(arraynp_output))
print (arraynp_output[3])

size of imageList_S2:   6
S2: (76, 125, 8)
MODIS (76, 125, 5)
CGLS (76, 125, 5)
S2: (76, 125, 8)
MODIS (76, 125, 5)
S2: (76, 125, 8)
MODIS (76, 125, 5)
S2: (76, 125, 8)
MODIS (76, 125, 5)
CGLS (76, 125, 5)
S2: (76, 125, 8)
MODIS (76, 125, 5)
S2: (76, 125, 8)
MODIS (76, 125, 5)
CGLS (76, 125, 5)
6
defaultdict(<class 'list'>, {'S2': [array([[[2.55000000e+02, 2.55000000e+02, 2.55000000e+02, ...,
         2.55000000e+02, 2.55000000e+02, 2.55000000e+02],
        [2.55000000e+02, 2.55000000e+02, 2.55000000e+02, ...,
         2.55000000e+02, 2.55000000e+02, 2.55000000e+02],
        [2.55000000e+02, 2.55000000e+02, 2.55000000e+02, ...,
         2.55000000e+02, 2.55000000e+02, 2.55000000e+02],
        ...,
        [2.55000000e+02, 2.55000000e+02, 2.55000000e+02, ...,
         2.55000000e+02, 2.55000000e+02, 2.55000000e+02],
        [2.55000000e+02, 2.55000000e+02, 2.55000000e+02, ...,
         2.55000000e+02, 2.55000000e+02, 2.55000000e+02],
        [2.55000000e+02, 2.55000000e+02, 2.55000000e+

In [17]:
outputCSV(arraynp_output, tilename, 'LAI')

**Multiple tiles LAI**

In [19]:
year_start = 2019
year_end = 2020
month_start = 6
month_end = 8
#tiles_CCRS = ee.FeatureCollection("users/ganghong/TilesSampling_Sub")
# feature1= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(0))
# feature2= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(1))
# feature3= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(2))
# feature4= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(3))
# feature1= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(4))
# feature2= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(5))
# feature3= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(6))
# feature4= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(7))
# feature5= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(8))
# feature6= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(9))
# feature7= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(10))
# feature8= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(11))
# feature9= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(12))
# feature10= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(13))
# feature11= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(14))
# feature12= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(15))
feature1= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(16))
feature2= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(17))
feature3= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(18))
feature4= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(19))
feature5= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(20))
feature6= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(21))
feature7= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(22))
feature8= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(23))
feature9= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(24))
feature10= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(25))
feature11= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(26))
feature12= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(27))
# feature1= ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(2))
# feature2=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(3))
# feature3=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(11))
# feature4=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(28))
# feature5=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(31))



newFeatCol= ee.FeatureCollection([feature1, feature2, feature3,feature4,feature5,feature6,feature7, feature8, feature9,feature10,feature11,feature12])
# newFeatCol= ee.FeatureCollection([feature1,feature2,feature3, feature4])
#print (newFeatCol)
# dataMODIS2 =ee.ImageCollection('MODIS/006/MCD15A3H')\
#                           .filterBounds(mapBounds_buf)\
#                           .filterDate(starDate, endDate)
for ii in range(0,newFeatCol.size().getInfo()):
    print (ii)
    mapBounds= ee.FeatureCollection(ee.Feature(newFeatCol.toList(newFeatCol.size()).get(ii))).geometry()   
    tilename=ee.Feature(newFeatCol.toList(newFeatCol.size()).get(ii)).get('MGRS_TILE')
    print('MGRS tilename:',tilename.getInfo())   
    #S2_input=S2_imgCol.filter(ee.Filter.calendarRange(year_start,year_end,'year')).filter(ee.Filter.calendarRange(month_start,month_end,'month')).filterMetadata('MGRS_TILE','equals',tilename)
    ## create a collection for S2 from GEE asset
    S2_imgCol=createCol(tilename)
    S2_input=S2_imgCol.filter(ee.Filter.calendarRange(year_start,year_end,'year')).filter(ee.Filter.calendarRange(month_start,month_end,'month'))
    ### run function applySensorComp
    coarseS2_output, coarseCGLS_output,coarseMODIS_output=applySensorComp(year_start,year_end, month_start, month_end, mapBounds, tilename, 'LAI', 4, S2_input,  1500, 'Lai','LaiStdDev', 'estimateLAI','errorLAI', LAI_CCRS,'b1','b2','b3')
    print (coarseS2_output.size().getInfo(), 'coarseS2_output')
    print ('coarseS2 band name:',coarseS2_output.first().bandNames().getInfo())  
    print (coarseCGLS_output.size().getInfo(), 'coarseCGLS_output')
    print (coarseMODIS_output.size().getInfo(), 'coarseMODIS_output')
    print (coarseS2_output.first().getInfo()['bands'][0], 'CoarseS2 band 0')
    print (coarseCGLS_output.first().getInfo()['bands'][0], 'Coarse CGLS band 0')
    print (coarseMODIS_output.first().getInfo()['bands'][0], 'Coarse MODIS band 0')
    coarsecrs=coarseS2_output.first().select(0).projection()
    coarsescale=coarseS2_output.first().select(0).projection().nominalScale()
    print (coarsecrs.getInfo(), 'Coarse S2 projection')
    print (coarsescale.getInfo(), 'Coarse S2 scale')
    arraynp_output=imageToarray('LAI',mapBounds, coarseS2_output, coarseCGLS_output, coarseMODIS_output)    
    print (len(arraynp_output))
    outputCSV(arraynp_output, tilename, 'LAI')
   # print (arraynp_output[2])

0
MGRS tilename: 13SCT
S2 size: 41
***In applySensorComp****
original MODIS band 0: {'id': 'LAI', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [484, 243], 'origin': [21714, 13358], 'crs': 'SR-ORG:6974', 'crs_transform': [463.312716528, 0, -20015109.354, 0, -463.312716527, 10007554.677]}
original MODIS scale: 463.3127165275
original dataCGLS band 0: {'id': 'LAI', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [405, 339], 'crs': 'EPSG:4326', 'crs_transform': [0.0029761904762040103, 0, -107.17410714252597, 0, -0.002976190476189799, 34.34077380953419]}
original dataCGLS scale: 331.3080083140298
original S2 size: [6568, 5595]
original S2 band 0: {'id': 'LAI', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -2147483648, 'max': 2147483647}, 'dimensions': [6568, 5595], 'crs': 'EPSG:4326', 'crs_transform': [0.0001796630568239043, 0, -107.1494301222651, 0, -0.0001796630568239043, 34.33792207

**Processing Fapar**

In [ ]:
# coarseS2_Fapar_output, coarseCGLS_Fapar_ouput,coarseMODIS_Fapar_output=applySensorComp('2019-04-01', '2019-09-30', mapBounds, tilename, 'fAPAR', 5, dataMODIS2, 500,'Fpar','FparStdDev', 'estimatefAPAR','errorfAPAR', fapar_CCRS,'b6','b1','b2')
# print (coarseS2_Fapar_output.size().getInfo(), 'coarseS2_Fapar_output')
# print (coarseCGLS_Fapar_ouput.size().getInfo(), 'coarseCGLS_Fapar_ouput')
# print (coarseMODIS_Fapar_output.size().getInfo(), 'coarseMODIS_Fapar_output')

In [29]:
year_start = 2019
year_end = 2020
month_start = 6
month_end = 8

# feature1= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(8))
# feature2= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(9))
# feature3= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(10))
# feature4= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(11))
# feature5= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(12))
# feature6= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(13))
# feature7= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(14))
# feature8= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(15))
# feature9= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(16))
# feature10= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(17))
feature11= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(18))
feature12= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(19))
feature13= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(20))
feature14= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(21))
feature15= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(22))
feature16= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(23))
feature17= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(24))
feature18= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(25))
feature19= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(26))
feature20= ee.Feature(tiles_CCRS.toList(tiles_CCRS.size()).get(27))
# feature1= ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(2))
# feature2=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(3))
# feature3=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(4))
# feature4=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(5))

# feature1= ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(12))
# feature2=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(14))
# feature3=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(6))
# feature4=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(7))
# feature5=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(11))
# feature1= ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(15))
# feature2=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(16))
# feature3=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(17))
# feature1= ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(18))
# feature2=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(20))
# feature3=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(22))
# feature1= ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(23))
# feature2=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(24))
# feature3=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(28))
# feature4=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(29))
# feature5=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(31))
# feature6=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(33))

# feature1= ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(0))
# feature2=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(1))
# feature3=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(8))
# feature4=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(9))

# feature1= ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(10))
# feature2=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(13))

# feature1=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(19))
# feature2=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(21))
# feature3=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(25))
# feature4=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(26))

# feature1= ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(32))
# feature2=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(34))

# newFeatCol= ee.FeatureCollection([feature1, feature2, feature3,feature4,feature5,feature6, feature7,feature8,feature9,feature10, feature11,feature12,feature13,feature14, feature15,feature16,feature17,feature18,feature19,feature20])
newFeatCol= ee.FeatureCollection([feature11,feature12,feature13,feature14, feature15,feature16,feature17,feature18,feature19,feature20])

# newFeatCol= ee.FeatureCollection([feature1, feature2, feature3,feature4])
# newFeatCol= ee.FeatureCollection([feature1, feature2])

for ii in range(0,newFeatCol.size().getInfo()):
    print (ii)
    mapBounds= ee.FeatureCollection(ee.Feature(newFeatCol.toList(newFeatCol.size()).get(ii))).geometry()   
    tilename=ee.Feature(newFeatCol.toList(newFeatCol.size()).get(ii)).get('MGRS_TILE')
    print('MGRS tilename:',tilename.getInfo())   
    #S2_input=S2_imgCol.filter(ee.Filter.calendarRange(year_start,year_end,'year')).filter(ee.Filter.calendarRange(month_start,month_end,'month')).filterMetadata('MGRS_TILE','equals',tilename)
    ## create a collection for S2 from GEE asset
    S2_imgCol=createCol(tilename)
    S2_input=S2_imgCol.filter(ee.Filter.calendarRange(year_start,year_end,'year')).filter(ee.Filter.calendarRange(month_start,month_end,'month'))
    ### run function applySensorComp
    #coarseS2_Fapar_output, coarseCGLS_Fapar_ouput,coarseMODIS_Fapar_output=applySensorComp('2019-04-01', '2019-09-30', mapBounds, tilename, 'fAPAR', 5, dataMODIS2, 500,'Fpar','FparStdDev', 'estimatefAPAR','errorfAPAR', fapar_CCRS,'b6','b1','b2')
    coarseS2_output, coarseCGLS_output,coarseMODIS_output=applySensorComp(year_start,year_end, month_start, month_end, mapBounds, tilename, 'fAPAR', 4, S2_input,  1500, 'Fpar','FparStdDev', 'estimatefAPAR','errorfAPAR', fapar_CCRS,'b6','b1','b2')
    print (coarseS2_output.size().getInfo(), 'coarseS2_output')
    print ('coarseS2 band name:',coarseS2_output.first().bandNames().getInfo())  
    print (coarseCGLS_output.size().getInfo(), 'coarseCGLS_output')
    print (coarseMODIS_output.size().getInfo(), 'coarseMODIS_output')
    print (coarseS2_output.first().getInfo()['bands'][0], 'CoarseS2 band 0')
    print (coarseCGLS_output.first().getInfo()['bands'][0], 'Coarse CGLS band 0')
    print (coarseMODIS_output.first().getInfo()['bands'][0], 'Coarse MODIS band 0')
    coarsecrs=coarseS2_output.first().select(0).projection()
    coarsescale=coarseS2_output.first().select(0).projection().nominalScale()
    print (coarsecrs.getInfo(), 'Coarse S2 projection')
    print (coarsescale.getInfo(), 'Coarse S2 scale')
    arraynp_output=imageToarray('fAPAR',mapBounds, coarseS2_output, coarseCGLS_output, coarseMODIS_output)    
    print (len(arraynp_output))
    outputCSV(arraynp_output, tilename, 'FAPAR')
   # print (arraynp_output[2])

0
MGRS tilename: 17SMU
S2 size: 31
***In applySensorComp****
original MODIS band 0: {'id': 'fAPAR', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [430, 240], 'origin': [26915, 13141], 'crs': 'SR-ORG:6974', 'crs_transform': [463.312716528, 0, -20015109.354, 0, -463.312716527, 10007554.677]}
original MODIS scale: 463.3127165275
original dataCGLS band 0: {'id': 'fAPAR', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [406, 336], 'crs': 'EPSG:4326', 'crs_transform': [0.0029761904762040103, 0, -82.09970238050718, 0, -0.002976190476189799, 35.24553571429589]}
original dataCGLS scale: 331.3080083140298
original S2 size: [6718, 5539]
original S2 band 0: {'id': 'fAPAR', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -2147483648, 'max': 2147483647}, 'dimensions': [6718, 5539], 'crs': 'EPSG:4326', 'crs_transform': [0.0001796630568239043, 0, -82.09972876153543, 0, -0.0001796630568239043, 35.243

In [ ]:
# for i in range (0,tiles_Bel.size().getInfo() ):
#     tilename=ee.Feature(tiles_Bel.toList(tiles_Bel.size()).get(i)).get('MGRS_TILE')
#     print (i, tilename.getInfo())